In [97]:
import pandas as pd
import numpy as np
import re 
from organizacao_fundos import df_tidy_simp

In [98]:
from unidecode import unidecode                   # só para fallback eventual
from organizacao_fundos import df_tidy_simp            # seu DataFrame financeiro
from mapear_codigo import ALIAS2CODE , CODIGOS_OFICIAIS, SUBCLASSES_OFICIAIS , normalizar

In [99]:
df_dash  = pd.read_excel("data/Estudo_de_Garantias_v3.xlsx",
                         sheet_name="Dashboard", header=1)

df_class = pd.read_excel("data/Estudo_de_Garantias_v3.xlsx",
                         sheet_name="Classificação", header=1)

# garantias já limpas/tokenizadas (colunas G1 … Gn)
df_tokens = pd.read_csv("data/garantias_cod.csv")
df_fin   = df_tidy_simp.copy()  

In [100]:
df_fin    = df_tidy_simp.reset_index(drop=False)   # preserva índice original
df_tokens = df_tokens.reset_index(drop=False)

In [101]:
df_all = (
    df_fin.merge(
        df_tokens.drop(columns=['Fundo', 'Ativo']),   # evita colunas duplicadas
        on='index',          # chave única garantida
        how='left',
        validate='1:1'
    )
    .drop(columns=['index']) # não precisamos mais dela
)

In [103]:
df_all_teste = df_all[df_all['Fundo'] == 'RURA11']  # substitua 'Fundo XYZ' pelo nome do fundo desejado
df_all_teste.head(60)  # exibe as primeiras linhas do DataFrame filtrado


,Fundo,%PL,Norm.,Ativo,Garantia,Nota,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12
380,RURA11,0.022,0.037543,NaN,AF Terras e Recebíveis,3.0,terras,AF,recebiveis/ credito/ direito creditorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381,RURA11,0.022,0.037543,NaN,Recebíveis,1.0,recebiveis/ credito/ direito creditorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,RURA11,0.021,0.035836,NaN,AF Terras,3.0,terras,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,RURA11,0.021,0.035836,NaN,Recebíveis,1.0,recebiveis/ credito/ direito creditorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,RURA11,0.021,0.035836,NaN,AF Terras,3.0,terras,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,RURA11,0.020,0.034130,NaN,AF Terras e Recebíveis,3.0,terras,AF,recebiveis/ credito/ direito creditorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,RURA11,0.020,0.034130,NaN,Recebíveis,1.0,recebiveis/ credito/ direito creditorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,RURA11,0.019,0.032423,NaN,AF Terras,3.0,terras,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,RURA11,0.019,0.032423,NaN,CDA/WA Estoques,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,RURA11,0.019,0.032423,NaN,AF Terras,3.0,terras,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_class['Subclasse_norm'] = df_class['Subclasse'].apply(normalizar)

class_map = (df_class
             .dropna(subset=['Subclasse_norm'])
             .set_index(['Código', 'Subclasse_norm'])['Nota']
             .to_dict())

In [105]:
SUB_NORM2CANON = {normalizar(s): s for s in SUBCLASSES_OFICIAIS}

In [106]:
SUB2CODE = {normalizar(row['Subclasse']): row['Código'].upper()
            for _, row in df_class.dropna(subset=['Subclasse']).iterrows()}

In [107]:
def classifica_token(tok: str):
    """
    Recebe um token bruto (já limpo) e devolve:
      - {"code": "AF"}              se for código ou alias de código
      - {"sub":  "Imóvel"}          se for subclasse oficial
      - {}                          caso contrário
    """
    if pd.isna(tok) or tok == "":
        return {}

    # 1) código oficial puro (duas ou três letras, etc.)
    if tok.upper() in CODIGOS_OFICIAIS:
        return {"code": tok.upper()}

    # 2) alias que mapeia para código
    tok_norm = normalizar(tok)
    if tok_norm in ALIAS2CODE:
        return {"code": ALIAS2CODE[tok_norm]}

    # 3) subclasse oficial
    if tok_norm in SUB_NORM2CANON:
        return {"sub": SUB_NORM2CANON[tok_norm]}

    # ruído
    return {}

In [108]:
def tokens_da_linha(row) -> tuple[list[str], list[str]]:
    """Extrai listas (codes, subs) a partir das colunas G1…Gn da linha."""
    codes, subs = [], []
    for col in row.index:
        if not col.startswith("G"):
            continue
        tok = row[col]
        info = classifica_token(tok)
        if "code" in info and info["code"] not in codes:
            codes.append(info["code"])
        if "sub"  in info and info["sub"]  not in subs:
            subs.append(info["sub"])
            
    # ★ INFERE código se subclasse tem mapeamento único
    for s in subs:
        cod_padrao = SUB2CODE.get(normalizar(s))
        if cod_padrao and cod_padrao not in codes:
            codes.append(cod_padrao)
    return codes, subs

In [109]:
def nota_para_linha(codes: list[str], subs: list[str]) -> float:
    """Escolhe a melhor nota possível para a combinação codes × subs."""
    notas = []
    for c in codes:
        for s in subs:
            notas.append(class_map.get((c, s), np.nan))
    notas_validas = [n for n in notas if not np.isnan(n)]

    if notas_validas:
        return float(np.nanmax(notas_validas))

    # fallback: pega a melhor nota do código ignorando subclasse
    notas_code_only = [
        v for (cod, sub), v in class_map.items()
        if cod in codes and not pd.isna(v)
    ]
    if notas_code_only:
        return float(np.nanmax(notas_code_only))

    return np.nan

In [110]:
def calculo_score(norm , nota_calculada):
    produto = norm * nota_calculada
    soma = produto.sum()
    return soma /0.03

In [111]:
codes_and_subs = df_all.filter(like="G").columns      # salva a lista de colunas G*

df_all[["codes", "subs"]] = (
    df_all[codes_and_subs]
      .apply(tokens_da_linha, axis=1, result_type="expand")
)

df_all["Nota_calculada"] = df_all.apply(
    lambda r: nota_para_linha(r["codes"], r["subs"]), axis=1
)

In [112]:
scores = (
    df_all.groupby("Fundo", sort=False)[["Norm.", "Nota_calculada"]]
          .apply(lambda g: calculo_score(g["Norm."], g["Nota_calculada"]))
)


In [113]:
print("─── Scores por Fundo ───")
for fundo, val in scores.items():
    print(f"{fundo}: {val:.2f}")

─── Scores por Fundo ───
VGIR11: 81.67
MXRF11: 78.64
RBRY11: 94.61
KNCR11: 80.60
RBRR11: 93.10
CPTR11: 41.96
KNCA11: 50.36
RURA11: 60.69
KNSC11: 76.72
BODB11: 61.08
KNUQ11: 46.89
HABT11: 71.02


In [114]:
scores_sorted = scores.sort_values(ascending=False).reset_index()
scores_sorted.columns = ["Ativo", "Score Garantia"]

# ——— duplica as colunas p/ layout lado‑a‑lado ———
df_out = pd.concat([scores_sorted, scores_sorted], axis=1)
df_out.columns = ["Ativo", "Score Garantia", "Ativo", "Score Garantia"]

# ——— grava planilha ———
df_out.to_excel("score_garantia_dashboard.xlsx", index=False)

print("Planilha salva como  score_garantia_dashboard.xlsx")
display(df_out.head(15))

Planilha salva como  score_garantia_dashboard.xlsx


,Ativo,Score Garantia,Ativo,Score Garantia
0,RBRY11,94.614858,RBRY11,94.614858
1,RBRR11,93.100041,RBRR11,93.100041
2,VGIR11,81.666410,VGIR11,81.666410
3,KNCR11,80.601421,KNCR11,80.601421
4,MXRF11,78.644323,MXRF11,78.644323
5,KNSC11,76.716532,KNSC11,76.716532
6,HABT11,71.018867,HABT11,71.018867
7,BODB11,61.084719,BODB11,61.084719
8,RURA11,60.693970,RURA11,60.693970
9,KNCA11,50.356473,KNCA11,50.356473


In [115]:
fundo_teste = "MXRF11"      # altere à vontade

df_debug = df_all[df_all["Fundo"] == fundo_teste]

display_cols = (
    ["Ativo", "Norm.", "Nota_calculada", "Nota", "codes", "subs"]
    + list(c for c in codes_and_subs)             # mostra G1…Gn
)

df_debug[display_cols].head(30)

,Ativo,Norm.,Nota_calculada,Nota,codes,subs,Garantia,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12
52,24C1526928,0.056734,1.0,1.0,"[A, CF, AF]",[recebiveis/ credito/ direito creditorio],- Aval\n- 60% da Cessão Fiduciária dos Recebív...,A,CF,recebiveis/ credito/ direito creditorio,A,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,24G1557250,0.052227,3.0,3.0,"[AF, FR]","[imovel, cotas de fundo (fip, fii etc.) e acoes]",- AF de Imóveis;\n- AF de Quotas;\n- Fundo de ...,AF,imovel,AF,"cotas de fundo (fip, fii etc.) e acoes",FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,23D1515316,0.039865,3.0,3.0,"[AF, F]",[imovel],- AF de imóvel\n- Fiança CSN,AF,imovel,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,22C0983841,0.037384,2.0,2.0,"[FR, AF]","[cotas de fundo (fip, fii etc.) e acoes]",- Fundo de Reserva;\n- Fundo de Despesas;\n- A...,FR,FR,AF,"cotas de fundo (fip, fii etc.) e acoes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,24A2246360,0.035658,3.0,3.0,"[AF, FR, F]",[imovel],- AF de imóvel;\n- Fundo de reserva;\n- Fundo ...,AF,imovel,FR,FR,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,24E2453531,0.031767,2.0,2.0,"[CF, A, FR, AF]",[spe],-Cessão de créditos e estoque/vendas futuras d...,CF,A,FR,FR,AF,spe,NaN,NaN,NaN,NaN,NaN,NaN
58,23H1074707,0.031688,1.0,2.0,"[AF, CF, F]",[recebiveis/ credito/ direito creditorio],- AF de quotas das SPEs;\n- CF de recebíveis;\...,AF,CF,recebiveis/ credito/ direito creditorio,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,20B0817201,0.029189,3.0,3.0,"[FR, F, AF, GL]","[banco, imovel]",- Fundo de 11 PMTs para o período de retrofit;...,FR,F,banco,AF,imovel,imovel,NaN,NaN,NaN,NaN,NaN,NaN
60,24E3191022,0.027178,3.0,3.0,"[AF, CF, FR]",[imovel],- AF de Imóvel;\n- Cessão fiduciária de direit...,AF,imovel,CF,FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,22L1314899,0.025879,3.0,3.0,[AF],"[cepac, cotas de fundo (fip, fii etc.) e acoes...",- Alienação Fiduciária de CEPAC; e/ou\n- Alien...,AF,cepac,AF,"cotas de fundo (fip, fii etc.) e acoes",AF,imovel,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df_all["Nota"] = pd.to_numeric(df_all["Nota"], errors="coerce")
df_all["Nota_calculada"] = pd.to_numeric(df_all["Nota_calculada"], errors="coerce")

# Filtra linhas em que Nota_calculada é diferente da Nota
df_diferencas = df_all[
    (df_all["Nota"].notna()) &
    (df_all["Nota_calculada"].notna()) &
    (df_all["Nota"] != df_all["Nota_calculada"])
]

# Seleciona apenas as colunas desejadas
colunas_desejadas = ["Fundo", "Ativo", "Garantia", "Nota", "Nota_calculada", "codes", "subs"]
df_filtrado = df_diferencas[colunas_desejadas]

# Exporta para Excel
df_filtrado.to_excel("notas_diferentes_limpo.xlsx", index=False)

print("Relatório salvo em 'notas_diferentes_limpo.xlsx'")

Relatório salvo em 'notas_diferentes_limpo.xlsx'
